In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
%cd /content/drive/MyDrive/A_Group/final_summary

/content/drive/MyDrive/BT4222_Group21


In [43]:
!pip install PyPDF2
!pip install ntlk

ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [44]:
import PyPDF2

def pdfToText(pdfFilePath, outputFilePath):
    print(f"Processing {pdfFilePath}")
    # Open the PDF file
    pdf_file = open(pdfFilePath, 'rb')

    # Create a PDF reader object
    reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF file
    num_pages = len(reader.pages)

    # Initialize a string variable to hold the text
    text = ""

    # Iterate through all the pages and extract text
    for page_num in range(num_pages):
        page_obj = reader.pages[page_num]
        text += page_obj.extract_text()

    # Close the PDF file
    pdf_file.close()

    # Open the file in write mode
    with open(outputFilePath, 'w') as f:
        # Write the text to the file
        f.write(text)

    # Print a message to confirm that the file has been saved
    print(f"Text saved to {outputFilePath}.")

In [45]:
import unicodedata
def remove_non_english_chars(text):
    # normalize text and check if ASCII value more than 128 (non-english words)
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn' and
                   ord(c) < 128)

In [46]:
import os
import re
import nltk
import pandas as pd
nltk.download('punkt')

if not os.path.exists("pdf_text"):
    os.makedirs("pdf_text")


input_filename = '../final_summary/2024_released_CGN_ESG_report.pdf'
output_filename = '../final_summary/pdf_text/CGN_2023.csv'
pdfToText(input_filename, output_filename)

# read the text file
with open(output_filename, 'r') as f:
        text = f.read()

# clean the data
text = remove_non_english_chars(text)
text = re.sub(r'\n', ' ', text)  # remove line breaks
text = re.sub(r'\s+', ' ', text)  # remove multiple spaces
sentences = nltk.sent_tokenize(text) # split the text into sentences
processed_sentences = []
for sentence in sentences:
    sentence = re.sub(r"\d+\n", "", sentence) # Remove page number
    sentence = re.sub(r"\n", " ", sentence) # Remove line breaks
    sentence = re.sub(r"\.(\s*\.){1,}", ". ", sentence) # Remove consecutive periods
    split_sentences = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)(?<!\d\.)\s*(?=[A-Z])", sentence) # split text into sentences

    split_sentences = [s for s in split_sentences if s != "" and s != "."] # do not include empty sentences or "."
    processed_sentences.extend(split_sentences)

    # create a dataframe
df = pd.DataFrame({'sentences': processed_sentences})
df.to_csv(output_filename, index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processing /content/drive/MyDrive/data/2024_released_CGN_ESG_report.pdf
Text saved to /content/drive/MyDrive/BT4222_Group21/pdf_text/CGN_2023.csv.


In [47]:
!pip install tensorflow
!pip install pytorch
!pip install torch
!pip install transformers

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)


### ESG-BERT
- Domain Specific BERT Model for Text Mining in Sustainable Investing
- URL: https://huggingface.co/nbroad/ESG-BERT
- This pre-trained model is able to classify each text and returns a label number which correlates to a textual label:
    * __label__Business_Ethics :  0
    * __label__Data_Security :  1
    * __label__Access_And_Affordability :  2
    * __label__Business_Model_Resilience :  3
    * __label__Competitive_Behavior :  4
    * __label__Critical_Incident_Risk_Management :  5
    * __label__Customer_Welfare :  6
    * __label__Director_Removal :  7
    * __label__Employee_Engagement_Inclusion_And_Diversity :  8
    * __label__Employee_Health_And_Safety :  9
    * __label__Human_Rights_And_Community_Relations :  10
    * __label__Labor_Practices :  11
    * __label__Management_Of_Legal_And_Regulatory_Framework :  12
    * __label__Physical_Impacts_Of_Climate_Change :  13
    * __label__Product_Quality_And_Safety :  14
    * __label__Product_Design_And_Lifecycle_Management :  15
    * __label__Selling_Practices_And_Product_Labeling :  16
    * __label__Supply_Chain_Management :  17
    * __label__Systemic_Risk_Management :  18
    * __label__Waste_And_Hazardous_Materials_Management :  19
    * __label__Water_And_Wastewater_Management :  20
    * __label__Air_Quality :  21
    * __label__Customer_Privacy :  22
    * __label__Ecological_Impacts :  23
    * __label__Energy_Management :  24
    * __label__GHG_Emissions :  25


In [48]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import math

tokenizer = AutoTokenizer.from_pretrained("nbroad/ESG-BERT")
model = AutoModelForSequenceClassification.from_pretrained("nbroad/ESG-BERT")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Step 1: Mapping ESG-BERT's topics to SGX and S&P ESG Criteria Topics
1. Using ESG-BERT to classify our SGX and S&P Global ESG Criteria Topics into its labels
2. We only take the predict label with highest probability.
3. For those labels without any SGX and S&P ESG Criteria Topics, we will manually map them ourselves based on relevancy

`labels_to_criteria`
- Key: ESG-BERT's ESG_Labels
- Value: SGX and S&P Global ESG Criteria Topics

In [49]:
import pandas as pd
import numpy as np
labels_to_criteria = {}
manual_mapping = []

# Criteria_Topics is a concatenated list of criteria topics from the industries that we are analyzing
with open("../final_summary/Scrapers/criteria_topics.txt", "r") as f:
    criteria_topics = eval(f.read())

for topic in criteria_topics:
    inputs = tokenizer(topic, return_tensors="pt")
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=1)
    top_probs, top_labels = torch.topk(probs, k=1)
    label = model.config.id2label[top_labels[0][0].item()]
    prob = top_probs[0][0].item()

    if label not in labels_to_criteria:
        labels_to_criteria[label] = []
    labels_to_criteria[label].append(topic)

# KEY: ESG-BERT's ESG_Labels
# Value: List of S&P Global ESG Criteria Topics
labels_to_criteria

{'Physical_Impacts_Of_Climate_Change': ['Climate Strategy'],
 'Business_Model_Resilience': ['Human Capital Development',
  'Innovation Management'],
 'Human_Rights_And_Community_Relations': ['Human Rights',
  'Environmental Policy & Management Systems',
  'Social Impacts on Communities',
  'Stakeholder Engagement'],
 'Data_Security': ['Information Security Cybersecurity & System Availability',
  'Network Reliability'],
 'Energy_Management': ['Operational Eco-Efficiency',
  'Low Carbon Strategy',
  'Energy Mix',
  'Electricity Generation',
  'Fuel Efficiency'],
 'Product_Design_And_Lifecycle_Management': ['Product Stewardship',
  'Packaging',
  'Sustainable Marketing & Brand Perception',
  'Sustainable Finance',
  'Circular Fashion'],
 'Supply_Chain_Management': ['Supply Chain Management'],
 'Employee_Engagement_Inclusion_And_Diversity': ['Talent Attraction & Retention',
  'Financial Inclusion'],
 'Labor_Practices': ['Labor Practice Indicators'],
 'Systemic_Risk_Management': ['Risk & Cr

For those ESG_labels that are not mapped to any ESG criteria topics, we would map them manually based on relevancy

In [50]:
labels_to_criteria = {
    'Physical_Impacts_Of_Climate_Change':['Climate Strategy'],
    'Board Composition': ['Women On The Board'],
    'Economic Performance': ['Total Assets', 'Net Profit'],
    'Business_Model_Resilience': ['Human Capital Development', 'Innovation Management'],
    'Human_Rights_And_Community_Relations': ['Human Rights', 'Environmental Policy & Management Systems', 'Social Impacts on Communities', 'Stakeholder Engagement','Employees Gender & Age Diversity'],
    'Data_Security': ['Information Security Cybersecurity & System Availability', 'Network Reliability'],
    'Energy_Management': ['Operational Eco-Efficiency', 'Low Carbon Strategy', 'Energy Mix', 'Electricity Generation', 'Fuel Efficiency', 'Water Related Risks'],
    'Product_Design_And_Lifecycle_Management': ['Product Stewardship', 'Packaging', 'Sustainable Marketing & Brand Perception', 'Sustainable Finance', 'Circular Fashion'],
    'Supply_Chain_Management': ['Supply Chain Management'],
    'Employee_Engagement_Inclusion_And_Diversity': ['Talent Attraction & Retention', 'Financial Inclusion'],
    'Labor_Practices': ['Labor Practice Indicators','Average Training Hours Per Employee'],
    'Systemic_Risk_Management': ['Risk & Crisis Management', 'Decarbonization Strategy'],
    'Product_Quality_And_Safety': ['Product Quality & Recall Management'],
    'Management_Of_Legal_And_Regulatory_Framework': ['Responsibility of Content', 'Compliance with Applicable Export Control Regimes'],
    'Competitive_Behavior': ['Corporate Governance', 'Market Opportunities'],
    'Employee_Health_And_Safety': ['Occupational Health & Safety', 'Health Outcome Contribution'],
    'Access_And_Affordability': ['Access to Healthcare'],
    'Business_Ethics': ['Business Ethics'],
    'Ecological_Impacts': ['Biodiversity', 'Sustainable Agricultural Practices'],
    'Waste_And_Hazardous_Materials_Management': ['Food Loss & Waste', 'Waste Generation'],
    'Water_And_Wastewater_Management': ['Water Related Risks'],
    'ESG_Certificates': ['List of ESG Certifications'],
    "Air_Quality": ['Low Carbon Strategy'], # manual mapping
    "Critical_Incident_Risk_Management": ['Risk & Crisis Management'], # manual mapping
    "Customer_Privacy": ['Information Security Cybersecurity & System Availability'], # manual mapping
    "Customer_Welfare": ['Social Impacts on Communities', 'Human Rights', 'Stakeholder Engagement'], # manual mapping
    "Director_Removal": ['Business Ethics'], # manual mapping
    "GHG_Emissions": ['Operational Eco-Efficiency', 'Green House Gas Emissions', 'Decarbonization Strategy'], # manual mapping
    "Selling_Practices_And_Product_Labeling": ['Business Ethics', 'Product Stewardship', 'Packaging', 'Sustainable Marketing & Brand Perception'], # manual mapping
}

### Step 2: Preparing all the mappings collected from webscrapping

Import Industry to Criteria Topics Mapping as `industry_criteria_mapping`
- Key: Industry
- Value: List of most relevant criteria topics

In [51]:
industry_criteria_df = pd.read_csv("../final_summary/Scrapers/Industry Criteria Topics.csv").transpose()
industry_criteria_mapping = {}

for industry, topics in industry_criteria_df.iterrows():
    if industry not in industry_criteria_mapping:
        industry_criteria_mapping[industry] = []
    topics = topics.dropna()
    industry_criteria_mapping[industry].extend(topics.to_list())
industry_criteria_mapping

{'Industry: New Energy': ['Climate Strategy',
  'Human Capital Development',
  'Human Rights',
  'Operational Eco-Efficiency',
  'Innovation Management',
  'Environmental Policy & Management Systems',
  'Women On The Board',
  'Supply Chain Management',
  'Water Related Risks',
  'Low Carbon Strategy',
  'Corporate Governance',
  'Business Ethics',
  'Occupational Health & Safety',
  'Labor Practice Indicators',
  'Waste Generation',
  'Green House Gas Emissions',
  'Employees Gender & Age Diversity',
  'Average Training Hours Per Employee',
  'List of ESG Certifications',
  'Economic Performance',
  'Total Assets',
  'Net Profit']}

Import the company to industry mappings as `company_industry_mapping` dictionary
- key: TICKER
- Value: Industry

### Step 3: Run report into ESG-BERT model to classify each sentence

In [52]:
import torch
import math

def run_model(input_file, model):
    counter = 1
    max_seq_length = tokenizer.model_max_length
    # with open(input_file, "r") as f:
    input_df = pd.read_csv(input_file)
    output = {}
    for line in input_df['sentences']:
        if len(line) <= max_seq_length:
            output['Line ' + str(counter)] = {}
            inputs = tokenizer(line, return_tensors="pt")
            outputs = model(**inputs)
            probs = outputs.logits.softmax(dim=1)

            # Extract the top 3 probabilities and labels
            top_probs, top_labels = torch.topk(probs, k=3)
            # store output
            output['Line ' + str(counter)]['Sentence'] = line
            for i in range(3):
                label = model.config.id2label[top_labels[0][i].item()]
                prob = top_probs[0][i].item()
                output['Line ' + str(counter)][f'ESG BERT Topic {i+1}'] = (label, prob)
            counter += 1
        else:
            # split input text into chunks
            text_chunks = []
            for i in range(math.ceil(len(line)/max_seq_length)):
                start = i * max_seq_length
                end = min((i+1)*max_seq_length, len(line))
                text_chunks.append(line[start:end])
            for chunk in text_chunks:
                output['Line ' + str(counter)] = {}
                inputs = tokenizer(chunk, return_tensors="pt", padding= True, truncation=True, max_length=max_seq_length)
                outputs = model(**inputs)
                probs = outputs.logits.softmax(dim=1)
                top_probs, top_labels = torch.topk(probs, k=3)

                output['Line ' + str(counter)]['Sentence'] = line
                for i in range(3):
                    label = model.config.id2label[top_labels[0][i].item()]
                    prob = top_probs[0][i].item()
                    output['Line ' + str(counter)][f'ESG BERT Topic {i+1}'] = (label, prob)
                counter += 1
    return output

### Step 4: Mapping output labels to our criteria topics
- We will map the output labels generated by ESG-BERT with the mapping, `mapped_to_criteria`
- We also have decided to classify sentences whose label with the highest probability is less than 0.50 as `Non-ESG`

In [53]:
''' [How it works]
    1. Get the list of most relevant criteria topics for the company based on its industry
    2. Based on the ESG_label with the highest probability, we get the list of potential criteria topics that can be mapped to the target ESG_label
    3. We check the list of potential criteria topics using the list of most relevant criteria topics
    4. If it exists, we just map the label to that criteria topic.
'''
def process_output(result, industry_criteria_mapping):
    # 将行业硬编码为“新能源”
    industry = "Industry: New Energy"

    # 初始化 DataFrame
    df = pd.DataFrame.from_dict(result, orient='index')

    # 获取“新能源”行业的相关 ESG 主题列表
    industry_criteria_topics = industry_criteria_mapping[industry]

    def mapperFunction(row, industry_criteria_topics):
        value = row[1]
        if value < 0.5:
            return ("NON-ESG", value)
        else:
            # 获取当前 ESG 标签的潜在映射主题
            mapped_criteria_topics = labels_to_criteria[row[0]]
            result = []
            # 检查是否符合“新能源”行业标准主题
            for topic in mapped_criteria_topics:
                if topic in industry_criteria_topics:
                    result.append(topic)
            return (result, value)
    df['Mapped Criteria Topic 1'] = df['ESG BERT Topic 1'].apply(lambda x: mapperFunction(x,industry_criteria_topics))
    df['Mapped Criteria Topic 2'] = df['ESG BERT Topic 2'].apply(lambda x: mapperFunction(x,industry_criteria_topics))
    df['Mapped Criteria Topic 3'] = df['ESG BERT Topic 3'].apply(lambda x: mapperFunction(x,industry_criteria_topics))
    return df

### Step 5: Export as CSV

In [54]:
def export_output_to_csv(df, output_filename):
    df.to_csv(output_filename)

### Step 6: Run all reports into the ESG-BERT Model

### NOTE: This will take about 2 hours to classify all 50 ESG reports
- For sample output, please run the commented code instead and comment out this chunk.

In [55]:
import os
import pandas as pd

'''
Note: This will take about 2 hours to classify all 25 ESG reports.

For a sample output please run the code chunk below
'''
input_file = output_filename
output_file = "../final_summary/classified_sentences/"+input_filename.split("/")[-1]

output_result = run_model(input_file, model)


In [56]:
output_df = process_output(output_result, industry_criteria_mapping)
export_output_to_csv(output_df, output_file)
print(f"Exported to {output_file}")

######## SAMPLE OUTPUT RUN THIS ########
# filename = "AMZN_2021.csv"
# ticker = filename.split("_")[0]
# industry = company_industry_mapping[ticker]
# input_file = "pdf_text" + "/" + filename
# ouptut_file = "output/classified_sentences/" + filename.replace(".csv", "_results.csv")
# output_result = run_model(input_file, model)
# output_df = process_output(output_result, industry_criteria_mapping, industry)
# export_output_to_csv(output_df, output_file)

Exported to ../BT4222_Group21/output/classified_sentences/2024_released_CGN_ESG_report.pdf


In [57]:
!pip install torch
!pip install transformers
!pip install evaluate
!pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [58]:
import transformers
import pandas as pd
import os
from collections import defaultdict
import re

In [59]:
from ast import literal_eval
topics_by_company = {}
reports = {}
df = pd.read_csv(output_file)
ticker = output_file.split('/')[-1].split('_')[0]
reports[ticker] = df

for company, data in reports.items():
    topics_by_company[company] = {}
    for index, row in data.iterrows():
        mapped_criteria_topic_1 = literal_eval(row['Mapped Criteria Topic 1'])
        mapped_criteria_topic_2 = literal_eval(row['Mapped Criteria Topic 2'])
        mapped_criteria_topic_3 = literal_eval(row['Mapped Criteria Topic 3'])
        sentence = row['Sentence']
        if mapped_criteria_topic_1[0] != 'NON-ESG':
            if mapped_criteria_topic_1[0] != []:
                for i in  mapped_criteria_topic_1[0]:
                    if i not in topics_by_company[company]:
                        topics_by_company[company][i] = {}
                    topics_by_company[company][i][sentence] = mapped_criteria_topic_1[1]
            else:
                if mapped_criteria_topic_2[0] != 'NON-ESG':
                    if mapped_criteria_topic_2[0] != []:
                        for i in  mapped_criteria_topic_2[0]:
                            if i not in topics_by_company[company]:
                                topics_by_company[company][i] = {}
                            topics_by_company[company][i][sentence] = mapped_criteria_topic_2[1]
                    else:
                        if mapped_criteria_topic_3[0] != 'NON-ESG':
                            if mapped_criteria_topic_3[0] != []:
                                for i in  mapped_criteria_topic_3[0]:
                                    if i not in topics_by_company[company]:
                                        topics_by_company[company][i] = {}
                                    topics_by_company[company][i][sentence] = mapped_criteria_topic_3[1]

In [60]:
# Sort sentences by topic according to probability
for company, data in topics_by_company.items():
    for topic, sentence_prob in data.items():
        sorted_sentence_prob = dict(sorted(sentence_prob.items(), key = lambda x: x[1], reverse=True))
        topics_by_company[company][topic] = sorted_sentence_prob

In [61]:
company_topic_sentences = {}
for company, data in topics_by_company.items():
    company_topic_sentences[company] = {}
    for topic, sentences in data.items():
        if topic not in company_topic_sentences[company]:
            company_topic_sentences[company][topic] = []
        for sentence, prob in sentences.items():
            company_topic_sentences[company][topic].append(sentence)

In [63]:
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Helper function to split the text into segments
def split_in_segments(text):
    tokens = 0
    mystring = list()
    segments = []
    text = re.sub(r'(?<=\w)(\.)', r' \1', text)
    for sent in sent_tokenize(text):
        newtokens = len(sent.split())
        if tokens + newtokens > 500:
            segments.append(" ".join(mystring))
            mystring = []
            mystring.append(str(sent).strip())
            tokens = newtokens
        elif newtokens > 500:
            segments.extend([str(sent)[i:i+500] for i in range(0, len(str(sent)), 500)])
        else:
            tokens += newtokens
            mystring.append(str(sent).strip())

    if mystring:
        segments.append(" ".join(mystring))

    return segments

# Helper Function to summarize text based on input text
def summarize_text(input_text):
    segments = split_in_segments(input_text)
    summaries = []
    for seg in segments:
        summary = summarizer(seg, truncation = True, max_length=100, min_length=30, do_sample=False)
        summaries.extend(summary)
    return summaries

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [64]:
company_summary_sentences = {}

for company, data in company_topic_sentences.items():
    print(f"Summarizing {company}")
    output = {}

    for topic, sentences in data.items():
        print(f"### Topic: {topic} ###")

        input_sentences = " ".join(sentences[:20]) # Top 20 sentences
        reference_summary = sentences[:40] # reference summary don't change
        summary_list = summarize_text(input_sentences)
        combined_summary = summary_list[0]['summary_text']
        for summary in summary_list[1:]:
            combined_summary += summary['summary_text']
        output[topic] = (combined_summary, " ".join(reference_summary))
    if company not in company_summary_sentences:
        company_summary_sentences[company] = {}
    company_summary_sentences[company] = output


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Summarizing 2024
### Topic: Human Capital Development ###


Your max_length is set to 100, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


### Topic: Innovation Management ###


Your max_length is set to 100, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


...

Your max_length is set to 100, but your input_length is only 43. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


### Topic: Human Rights ###
### Topic: Environmental Policy & Management Systems ###
### Topic: Employees Gender & Age Diversity ###
### Topic: Operational Eco-Efficiency ###
### Topic: Low Carbon Strategy ###
### Topic: Water Related Risks ###
### Topic: Business Ethics ###
### Topic: Corporate Governance ###
### Topic: Green House Gas Emissions ###
### Topic: Supply Chain Management ###
### Topic: Waste Generation ###
### Topic: Labor Practice Indicators ###


Your max_length is set to 100, but your input_length is only 73. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)


### Topic: Average Training Hours Per Employee ###


Your max_length is set to 100, but your input_length is only 73. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)


In [65]:
output_df = pd.DataFrame.from_dict(output, orient='index')
output_df = output_df.rename_axis("topic").reset_index()
output_df.rename(columns={0: "combined_summary", 1: "reference_summary"}, inplace=True)
output_df.to_csv(f"../final_summary/Generated Summaries/{company}_top20_summaries.csv")

In [66]:
# Helper Function to compute ROUGE-N scores from Huggingface's evaluate library
import evaluate
def evaluateRouge(output):
    output_results = {}
    rouge = evaluate.load('rouge')
    for idx, row in output.iterrows():
        topic = row[0]
        generated = [row[1]]
        reference = [row[2]]
        results = rouge.compute(predictions=generated, references = reference)
        output_results[topic] = results
    return output_results

In [67]:
# Helper Function to calculate the average ROUGE-N score
def averageEvaluation(results):
    output_dict = {
        'rouge1': 0,
        'rouge2': 0,
        'rougeL': 0,
        'rougeLsum': 0
    }
    for topic, rougeData in results.items():
        output_dict['rouge1'] = sum(val['rouge1'] for val in results.values())/len(results)
        output_dict['rouge2'] = sum(val['rouge2'] for val in results.values())/len(results)
        output_dict['rougeL'] = sum(val['rougeL'] for val in results.values())/len(results)
        output_dict['rougeLsum'] = sum(val['rougeLsum'] for val in results.values())/len(results)
    return output_dict

In [68]:
!pip install rouge_score

In [69]:
# For each company, compute average ROUGE-N scores across all generated summaries
import os
generated_summaries_path = "../final_summary/Generated Summaries/"

evaluation_results = {}

for filename in os.listdir(generated_summaries_path):
    summaries = pd.read_csv(generated_summaries_path + filename)
    summaries_result = evaluateRouge(summaries)
    company = filename.split("_")[0]
    input_type = filename.split("_")[1]
    if input_type not in evaluation_results:
        evaluation_results[input_type] = {}
    evaluation_results[input_type][company] = averageEvaluation(summaries_result)

<ipython-input-66-9d74cd3c5669>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  topic = row[0]
<ipython-input-66-9d74cd3c5669>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  generated = [row[1]]
<ipython-input-66-9d74cd3c5669>:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  reference = [row[2]]
<ipython-input-66-9d74cd3c5669>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, inte

In [70]:
# Save evaluation data as pickle file to reopen for review
import pickle

save_file = True # Set as true if you want to save the evaluation_results dictionary as pickle file

if save_file: # save evaluation results as pickle file
    file_to_save = open('Evaluation_Result', 'ab')
    pickle.dump(evaluation_results, file_to_save)
    file_to_save.close()
else: # Set as false to open the file
    saved_file = open('Evaluation_Result', 'rb')
    evaluation_results = pickle.load(saved_file)

In [71]:
# Calculate Average ROUGE-N scores across types of inputs (Top20/Top30/Random20/Random30)
show_evaluation = {}
for k,v in evaluation_results.items():
    show_evaluation[k] = {
        'rouge1': 0,
        'rouge2': 0,
        'rougeL': 0,
        'rougeLsum': 0
    }

    show_evaluation[k]['rouge1'] = sum(val['rouge1'] for val in v.values())/len(v)
    show_evaluation[k]['rouge2'] = sum(val['rouge2'] for val in v.values())/len(v)
    show_evaluation[k]['rougeL'] = sum(val['rougeL'] for val in v.values())/len(v)
    show_evaluation[k]['rougeLsum'] = sum(val['rougeLsum'] for val in v.values())/len(v)

pd.DataFrame.from_dict(show_evaluation, orient='index').sort_index()

,rouge1,rouge2,rougeL,rougeLsum
top15,0.010351,0.002196,0.009916,0.009916
top20,0.098327,0.089530,0.092187,0.092187
top25,0.008304,0.001392,0.007948,0.007948
top30,0.007508,0.001335,0.007178,0.007178


In [74]:
import openai
import pandas as pd

# 设置 OpenAI API 密钥
openai.api_key = 'sk-vzRrOeVbcm0RfRvb7k29Sy13PNAqCdChUvmvaDVs4iT3BlbkFJZcMi-v0cfYyqyTzr9xfM0QIk9Lk_8vPkdbCGaHxKUA'

import openai
import pandas as pd

# 读取CSV文件
csv_file_path = f"../final_summary/Generated Summaries/{company}_top20_summaries.csv"
df = pd.read_csv(csv_file_path)

# 定义用于调用 ChatGPT API 的函数
def get_quantitative_summary(text):
    # Use client.chat.completions.create with OpenAI object
    client = openai.OpenAI(api_key='sk-vzRrOeVbcm0RfRvb7k29Sy13PNAqCdChUvmvaDVs4iT3BlbkFJZcMi-v0cfYyqyTzr9xfM0QIk9Lk_8vPkdbCGaHxKUA',) # Create an OpenAI object
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", # Use a suitable chat model, like gpt-3.5-turbo
        messages=[
            {"role": "system", "content": "You are a helpful ESG expert that summarizes a company's ESG report text."},
            {"role": "user", "content": f"Please summarize the following text in 1-2 sentences containing quantitative explanation if available:\n{text}"}
        ],
        temperature=0.3,  # 保持生成内容的稳定性
        max_tokens=500,
        top_p=1.0,       # 使用完整的采样
        frequency_penalty=0,  # 允许适度重复关键信息
        presence_penalty=0
    )
    summary = response.choices[0].message.content # Access the summary text from the response
    return summary.strip()

# 存储每行处理后的量化摘要
summary_sentences = []

# 遍历每行，调用 API 生成量化的1-2句总结
for index, row in df.iterrows():
    topic = row['topic']
    combined_summary = row['combined_summary']

    # 调用 ChatGPT API 生成量化总结
    quantitative_summary = get_quantitative_summary(combined_summary)

    # 将量化总结添加到列表中
    summary_sentences.append(f"{topic}:{quantitative_summary}\n")

# 拼接成完整的公司级摘要
final_summary = " ".join(summary_sentences)

# 打印公司级摘要
print(final_summary)

Human Capital Development:The company emphasizes market awareness through business strategy, risk assessment, and transparent communication with stakeholders, including monthly power generation updates and major project disclosures. They are committed to enhancing ESG reporting and partnering with sustainable business allies.
 Innovation Management:The company emphasizes market awareness through its business strategy, market opportunities, and marketing efforts. They assess risks in alignment with business objectives, strategic focus, and performance-based assessment. The company also provides regular updates on power generation and discloses major project investments to meet market expectations and improve transparency. They commit to enhancing ESG reporting to meet stakeholder expectations and aim to collaborate with partners who share their values and commitment to sustainability.
 Occupational Health & Safety:CGN New Energy Holdings Co., Ltd. emphasizes occupational health and safe

Text Topic

In [ ]:
!pip install pdfplumber PyPDF2
! pip install transformers
import datetime
!pip install sentencepiece accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration